In [27]:
#imports
import numpy as np
import scipy.stats as stats
import os

### Feature extraction
train will have the training samples and test will have the test samples

In [254]:
#speaker dependent partitions
#sdeptrain dimensions sdeptrain[speaker][word][utterance idx] with idxs given alphabetically
sdeptrain = [[[[] for i in range(4)] for j in range(5)] for k in range(4)]
#sdeptest dimensions sdeptrain[speaker][word] with idxs given alphabetically
sdeptest = [[[] for i in range(5)] for j in range(4)]
p = './feature'
speakerList = ['dg', 'ls', 'mh', 'yx']
wordList = ['asr','cnn','dnn','hmm','tts']
snum = 0
for s in speakerList:
    speakerpath = p+'/'+s
    wnum = 0
    for w in wordList:
        wordpath = speakerpath +'/'+s+'_'+w
        #set up train
        for utt in range(1,5):
            fpath = wordpath + str(utt)+'.fea'
            f = open(fpath,'r')
            data = f.readlines()
            data = [i.split(',') for i in data]
            for sample in range(len(data)):
                data[sample] = [float(i) for i in data[sample]]
            for d in data:
                sdeptrain[snum][wnum][utt-1].append(d)
        #set up test
        fpath2 = wordpath + str(5)+'.fea'
        f2 = open(fpath2,'r')
        data = f2.readlines()
        data = [i.split(',') for i in data]
        for d in data:
            sdeptest[snum][wnum].append(d)
        wnum += 1
    snum += 1
sdeptrain = np.array(sdeptrain)

sdepasr_mn = np.mean([[np.mean(np.array(sdeptrain[i][0][j]),axis=0)] for i in range(4) for j in range(4)],axis=0)[0]
sdepcnn_mn = np.mean([[np.mean(np.array(sdeptrain[i][1][j]),axis=0)] for i in range(4) for j in range(4)],axis=0)[0]
sdepdnn_mn = np.mean([[np.mean(np.array(sdeptrain[i][2][j]),axis=0)] for i in range(4) for j in range(4)],axis=0)[0]
sdephmm_mn = np.mean([[np.mean(np.array(sdeptrain[i][3][j]),axis=0)] for i in range(4) for j in range(4)],axis=0)[0]
sdeptts_mn = np.mean([[np.mean(np.array(sdeptrain[i][4][j]),axis=0)] for i in range(4) for j in range(4)],axis=0)[0]
bigasr = []
bigcnn = []
bigdnn = []
bighmm = []
bigtts = []
for i in range(4):
    for j in range(4):
        bigasr.extend(sdeptrain[i][0][j])
        bigcnn.extend(sdeptrain[i][1][j])
        bigdnn.extend(sdeptrain[i][2][j])
        bighmm.extend(sdeptrain[i][3][j])
        bigtts.extend(sdeptrain[i][4][j])
sdepasr_cov = np.cov(np.array(bigasr).T)
sdepcnn_cov = np.cov(np.array(bigcnn).T)
sdepdnn_cov = np.cov(np.array(bigdnn).T)
sdephmm_cov = np.cov(np.array(bighmm).T)
sdeptts_cov = np.cov(np.array(bigtts).T)
print(sdepasr_cov)

[[ 1.04048341e+01  1.27636270e+01  2.43505882e-02 -9.68879574e-02
   6.62948182e-01 -4.48165841e-01 -5.90144298e-01 -4.66678538e-01
  -8.21344077e-02 -1.66746807e-01  1.62683654e-01  2.05282523e-02
   2.98959372e-01  2.53225667e-01]
 [ 1.27636270e+01  1.87229059e+01 -7.55863487e-01 -7.56105813e-01
   8.98603059e-01 -7.48330443e-01 -7.91945381e-01 -6.17434456e-01
  -3.50967201e-01 -3.93730449e-01 -7.84939754e-02  6.66048585e-02
   3.80946887e-01  3.67717591e-01]
 [ 2.43505882e-02 -7.55863487e-01  2.43649242e+00 -8.18914427e-01
   8.31471700e-02 -2.26633800e-03 -5.55522437e-02 -1.32992328e-02
   9.70252900e-02  1.23088858e-01  1.04846005e-01  2.44834159e-02
   8.68224494e-02 -5.60005611e-02]
 [-9.68879574e-02 -7.56105813e-01 -8.18914427e-01  1.00768454e+00
  -5.12910447e-02 -4.05244669e-02  3.36360552e-02  8.31173113e-02
   1.97579194e-02 -7.15650203e-02  6.20710968e-02 -8.21601356e-03
  -1.73139381e-02  1.65536484e-03]
 [ 6.62948182e-01  8.98603059e-01  8.31471700e-02 -5.12910447e-02
  

In [255]:
# print(sdeptrain[0][0])

In [256]:
#speaker independent partitions
#Extract feature vectors
p = './feature'
#test separetes each utterance of each word of each speaker
test = []
train = []
#test_big chunks all utterances of each word for each speaker
test_big = []
train_big = []
dirList = ['dg', 'ls', 'mh', 'yx']
for d in os.listdir(p):
    #put mh in the test set
    if d == 'mh':
        dirpath = p + '/' + d
        filelist = os.listdir(dirpath)
        word = [[] for i in range(5)]
        word_big = [[] for i in range(5)]
        for i in range(len(filelist)):
            f = open(dirpath + '/' + filelist[i])
            data_ = f.readlines() 
            data = [i.split(',') for i in data_]
            for sample in range(len(data)):
                data[sample] = [float(s) for s in data[sample] ]
            word[i//5].append(data)
            word_big[i//5].extend(data)
        test.append(word)
        test_big.append(word_big)
    else:
        if d in dirList:
            #put all others in train
            dirpath = p + '/' + d
            filelist = os.listdir(dirpath)
            word = [[] for i in range(5)]
            word_big = [[] for i in range(5)]
            for i in range(len(filelist)):
                f = open(dirpath + '/' + filelist[i])
                data_ = f.readlines() 
                data = [i.split(',') for i in data_]
                for sample in range(len(data)):
                    data[sample] = [float(s) for s in data[sample] ]
                word[i//5].append(data)
                word_big[i//5].extend(data)
            train.append(word)
            train_big.append(word_big)

In [257]:
print(len(train_big),len(train_big[0]))

3 5


In [258]:
#speaker dependent hmm

print(np.exp(1))

2.718281828459045


In [259]:
a = [[1,1,1],
     [2,2,2],
     [3,3,3]]

a_ = [a[-i] for i in range(1,len(a)+1)]
print(a_)

[[3, 3, 3], [2, 2, 2], [1, 1, 1]]


In [263]:
##### HELPER FUNCTIONSSSSS
#implemented from https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm

#Forward Pass

def calculateBMatrix(X, mu, sigma,N):
    #X is the data in a file
    #mu is the average across all states 
    #sigma is the covariance matrix across all states 
    #N is the number of states
    #BMatrix = [T,N]

    T = len(X)
    bMatrix = np.zeros((N,T))
    for frame in range(T):
        temp_frame = []
        for state in range(N):
            bMatrix[state,frame] = stats.multivariate_normal(mean=(mu[state]),cov=sigma[state]).pdf(X[frame])
        
    return bMatrix

def calculateAlphaMatrix(transMatrix, BMatrix,priors):
    #transMatrix is A 
    #BMatrix is B
    #priors is pi
    #populate our alpha matrix
    T = len(BMatrix[0])
    N = len(BMatrix)
    Amat = np.array(transMatrix)
    alpha = np.zeros((N,T))
    #alphaMatrix = [T,N]
    for i in range(N):
        alpha[i,0]=priors[i]*BMatrix[i,0]
    for t in range(1,T):
        for i in range(0,N):
            alpha[i,t]=BMatrix[i,t]*np.inner(alpha[:,t-1],Amat[:,i])
        
#     alphaMatrix = []

#     for frame in range(T):
#         temp_frame = []
#         for state in range(N):
#             if frame == 0:
#                 #if we are in the first frame
#                 temp_frame.append(priors[state] * BMatrix[frame][state])
#             else:
#                 #any other frame
#                 s = sum([ alphaMatrix[frame-1][n] * transMatrix[n][state] for n in range(N)])
#                 x = BMatrix[frame][state] * s
#                 temp_frame.append(x)
#         alphaMatrix.append(temp_frame)
        
    return alpha
       
#Backward Pass

def calculateBetaMatrix(transMatrix, bMatrix):
    #populate the Beta matrix

    Amat = np.array(transMatrix)
    Bmat = np.array(bMatrix)
    T = len(bMatrix[0])
    N = len(bMatrix)
    beta = np.zeros((N,T))
    for i in range(N):
        beta[i,T-1]=1
    for t in range(T-2,-1,-1):
        for i in range(0,N):
            beta[i,t]=np.inner(Amat[i,0:N],beta[:,t+1]*Bmat[:,t+1])
    return beta
#Update step helper functions

def calculateGammaMatrix(alphaMatrix, betaMatrix):
    T = len(betaMatrix[0])
    N = len(betaMatrix)
    gamma = np.zeros((N,T))
    for t in range(T):
        gamma[:,t]=alphaMatrix[:,t]*betaMatrix[:,t]
        gamma[:,t]=gamma[:,t]/np.sum(gamma[:,t])
#     sumList = np.zeros((T))
#     for f in range(T):
#         for n in range(N):
#             sumList[f] += alphaMatrix[f][n]*betaMatrix[f][n]
#     #sumList = [sum([alphaMatrix[f][n] * betaMatrix[f][n] for n in range(N)]) for f in range(T)]
#     gammaMatrix = []
#     for frame in range(T):
#         temp_frame = []
#         for state in range(N):
#             temp_frame.append(alphaMatrix[frame][state] / sumList[frame])
#         gammaMatrix.append(temp_frame)
    return gamma

def calculateXiMatrix( transMatrix, alphaMatrix, betaMatrix, bMatrix):
    #caluculate xi 
    #probability of going from state n to state j at times t and t+1 respectively
    T = len(bMatrix)
    N = len(bMatrix[0])
    
    xiMatrix = [[0 for i in range(2*N)] for j in range(T)]
    
    for frame in range(T):
        for state in range(N):
            for state_ in range(state, state + 2):
                xiMatrix[frame, state + state_] = alphaMatrix[frame, state] * transMatrix[state][state_]
                if frame < (T-1):
                    if state_ == N:
                        xiMatrix[frame, state + state_] = 0
                    else:
                        xiMatrix[frame, state + state_] = xiMatrix[frame, state + state_] * bMatrix[frame + 1, state_] * betaMatrix[frame+1, state_]
        xi[:,t]=xi[:,t]/np.sum(xi[:,t])
        xiMatrix[frame,:] = xiMatrix[frame,:]/np.sum(xiMatrix[frame, :]) 
    
    return xiMatrix 

#Update steps
def updatePriors(gammaMatrix):
    #we want the expected frequency spent in state i at time step t = 1
    new_priors = []
    N = len(gammaMatrix[0])
    for n in range(N):
        new_priors.append(gammaMatrix[0][n])
    return new_priors

def updateA(priors, transMatrix, X, mu, sigma, n, j, t, N, T):
    num = 0
    for i in range(T-1):
        #across all t
        temp = calculateXi(transMatrix, X, mu, sigma, n, j, i, N, T)
        num += temp
        
    denom = 0
    for i in range(T-1):
        #across all t
        temp = calculateGamma(priors, transMatrix, X, mu, sigma, n, i, N, T)
        denom += temp
    
    alpha_update = num / denom
    
    return alpha_update

def updateB(vk, priors, transMatrix, X, mu, sigma, n, N, T):
    #vk is a specific observation
    #how to compare that against all the y_t?
    num = 0
    for i in range(T):
        #if yt == vk:
        temp = calculateGamma(priors, transMatrix, X, mu, sigma, n, i, N, T)
        num += temp
    denom = 0
    for i in range(T):
        temp = calculateGamma(priors, transMatrix, X, mu, sigma, n, i, N, T)
        denom += temp
    b_update = num / denom
    
    return b_update

In [264]:
#speaker independent hmm
initialProbabilities = [0.2, 0.2, 0.2, 0.2, 0.2]

initialTransition = [[0.8,0.2,  0,  0,   0],
                     [  0,0.8,0.2,  0,   0],
                     [  0,  0,0.8,0.2,   0],
                     [  0,  0,  0,0.8, 0.2],
                     [  0,  0,  0,  0,   1]]

#create an HMM for each word
word0 = []
word1 = []
word2 = []
word3 = []
word4 = []

for speaker in range(len(train_big)):
    for word in range(len(train_big[speaker])):
        if word == 0:
            word0.extend(train_big[speaker][word])
        if word == 1:
            word1.extend(train_big[speaker][word])
        if word == 2:
            word2.extend(train_big[speaker][word])
        if word == 3:
            word3.extend(train_big[speaker][word])
        if word == 4:
            word4.extend(train_big[speaker][word])

#initialize mean of hmm for the first word
word0 = np.array(word0)
#mu should be (1x14) for each state
# mu_matrix
mu0 = np.mean(word0, axis=0)
#initialize the covariance matrix, sigma, of the hmm for the first word
word0_np = np.array(word0)
#covmat should be (14x14) for each state
covmat0 = np.cov(word0_np.T)

# Baum Welch this bitchhhhhh



Comments

There are N = 5 possible states and the number of observations is equal to the number of rows or time steps for each utterance. At each time step or observation, we are in one of the states, so a number 1-5 i think. So T = how many ever rows there are present. Still not sure exactly how we should chunk the data. Not sure if we should separate it by utterance or if we should group all the utterances of the same word together. Update parameters using baum welch forward/backward propogation equations. According to Piazza, update alpha and beta after each individual utterance and update the transition matrix after all the files for that word? Not how to implement this though. 

We train one hmm per word, so 5 in total. To predict a test sample, we run the utterance through each of the 5 models and assign it the label of the hmm that gives the highest probability.

In [265]:
#SDEP for ASR
A = initialTransition
mn = [sdepasr_mn,sdepasr_mn,sdepasr_mn,sdepasr_mn,sdepasr_mn]
cov = [sdepasr_cov,sdepasr_cov,sdepasr_cov,sdepasr_cov,sdepasr_cov]
B = calculateBMatrix(sdeptrain[0][0][0],mn,cov,5)
Alpha = calculateAlphaMatrix(A,B,initialProbabilities)
Beta = calculateBetaMatrix(A,B)
Gamma = calculateGammaMatrix(Alpha,Beta)
print(Gamma)

[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan 

/home/kalinuxmsi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in true_divide
